In [39]:
import numpy as np
import pandas as pd
import re 
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize


from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix,classification_report

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential,Model
from tensorflow.keras.layers import Embedding, Conv1D,Input, GlobalMaxPooling1D,MaxPooling1D, Dense



In [40]:
data = pd.read_csv('IMDB-Dataset.csv',encoding='ISO-8859-1')
print(data.shape)
data.head()

(50000, 2)


,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [41]:
data.sentiment.replace('positive',1,inplace=True)
data.sentiment.replace('negative',0,inplace=True)
data.head(10)

,review,sentiment
0,One of the other reviewers has mentioned that ...,1
1,A wonderful little production. <br /><br />The...,1
2,I thought this was a wonderful way to spend ti...,1
3,Basically there's a family where a little boy ...,0
4,"Petter Mattei's ""Love in the Time of Money"" is...",1
5,"Probably my all-time favorite movie, a story o...",1
6,I sure would like to see a resurrection of a u...,1
7,"This show was an amazing, fresh & innovative i...",0
8,Encouraged by the positive comments about this...,0
9,If you like original gut wrenching laughter yo...,1


1. Remove HTML tags

In [42]:
def clean(text):
    cleaned = re.compile(r'<.*?>')
    return re.sub(cleaned,'',text)

data.review = data.review.apply(clean)
data.review[0]

"One of the other reviewers has mentioned that after watching just 1 Oz episode you'll be hooked. They are right, as this is exactly what happened with me.The first thing that struck me about Oz was its brutality and unflinching scenes of violence, which set in right from the word GO. Trust me, this is not a show for the faint hearted or timid. This show pulls no punches with regards to drugs, sex or violence. Its is hardcore, in the classic use of the word.It is called OZ as that is the nickname given to the Oswald Maximum Security State Penitentary. It focuses mainly on Emerald City, an experimental section of the prison where all the cells have glass fronts and face inwards, so privacy is not high on the agenda. Em City is home to many..Aryans, Muslims, gangstas, Latinos, Christians, Italians, Irish and more....so scuffles, death stares, dodgy dealings and shady agreements are never far away.I would say the main appeal of the show is due to the fact that it goes where other shows wo

3. Convert everything to lowercase

In [43]:
def to_lower(text):
    return text.lower()

data.review = data.review.apply(to_lower)
data.review[0]

"one of the other reviewers has mentioned that after watching just 1 oz episode you'll be hooked. they are right, as this is exactly what happened with me.the first thing that struck me about oz was its brutality and unflinching scenes of violence, which set in right from the word go. trust me, this is not a show for the faint hearted or timid. this show pulls no punches with regards to drugs, sex or violence. its is hardcore, in the classic use of the word.it is called oz as that is the nickname given to the oswald maximum security state penitentary. it focuses mainly on emerald city, an experimental section of the prison where all the cells have glass fronts and face inwards, so privacy is not high on the agenda. em city is home to many..aryans, muslims, gangstas, latinos, christians, italians, irish and more....so scuffles, death stares, dodgy dealings and shady agreements are never far away.i would say the main appeal of the show is due to the fact that it goes where other shows wo

4. Remove stopwords

In [44]:
def rem_stopwords(text):
    stop_words = set(stopwords.words('english'))
    words = word_tokenize(text)
    return [w for w in words if w not in stop_words]

data.review = data.review.apply(rem_stopwords)


1. Creating Bag Of Words (BOW)

In [45]:
data.shape

(50000, 2)

In [46]:
X = np.array(data.iloc[:,0].values)
y = np.array(data.sentiment.values)
print("X.shape = ",X.shape)
print("y.shape = ",y.shape)

X.shape =  (50000,)
y.shape =  (50000,)


2. Train Test Split

In [47]:
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.3)

print("Train shapes : X = {}, y = {}".format(X_train.shape,y_train.shape))
print("Test shapes : X = {}, y = {}".format(X_test.shape,y_test.shape))


Train shapes : X = (35000,), y = (35000,)
Test shapes : X = (15000,), y = (15000,)


3. Tokenizing

In [48]:
from tensorflow.keras.preprocessing.text import Tokenizer
max_words=10000
tokenizer=Tokenizer(max_words)
tokenizer.fit_on_texts(X_train)
X_train=tokenizer.texts_to_sequences(X_train)
X_test=tokenizer.texts_to_sequences(X_test)


In [49]:
word2vec=tokenizer.word_index
V=len(word2vec)
print('dataset has %s number of independent tokens' %V)

dataset has 162815 number of independent tokens


In [50]:

X_train=pad_sequences(X_train)
X_train.shape

(35000, 1377)

In [51]:
T=X_train.shape[1]
X_test=pad_sequences(X_test,maxlen=T)
X_test.shape

(15000, 1377)

In [60]:

D = 20

model = Sequential()
model.add(Embedding(V+1, D, input_shape=(T,)))
model.add(Conv1D(32, 3, activation='relu'))
model.add(MaxPooling1D(3))
model.add(Conv1D(64, 3, activation='relu'))
model.add(MaxPooling1D(3))
model.add(Conv1D(128, 3, activation='relu'))
model.add(GlobalMaxPooling1D())
model.add(Dense(5, activation='softmax'))

model.summary()


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_5 (Embedding)      (None, 1377, 20)          3256320   
_________________________________________________________________
conv1d_15 (Conv1D)           (None, 1375, 32)          1952      
_________________________________________________________________
max_pooling1d_10 (MaxPooling (None, 458, 32)           0         
_________________________________________________________________
conv1d_16 (Conv1D)           (None, 456, 64)           6208      
_________________________________________________________________
max_pooling1d_11 (MaxPooling (None, 152, 64)           0         
_________________________________________________________________
conv1d_17 (Conv1D)           (None, 150, 128)          24704     
_________________________________________________________________
global_max_pooling1d_3 (Glob (None, 128)               0

In [61]:
model.compile(loss='sparse_categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
cnn_senti=model.fit(X_train,y_train,epochs=10,batch_size=100)

Epoch 1/10
350/350 [==============================] - 8s 19ms/step - loss: 0.5526 - accuracy: 0.6911
Epoch 2/10
350/350 [==============================] - 6s 19ms/step - loss: 0.2525 - accuracy: 0.8987
Epoch 3/10
350/350 [==============================] - 6s 19ms/step - loss: 0.1691 - accuracy: 0.9386
Epoch 4/10
350/350 [==============================] - 7s 19ms/step - loss: 0.1097 - accuracy: 0.9615
Epoch 5/10
350/350 [==============================] - 7s 19ms/step - loss: 0.0596 - accuracy: 0.9817
Epoch 6/10
350/350 [==============================] - 7s 19ms/step - loss: 0.0256 - accuracy: 0.9929
Epoch 7/10
350/350 [==============================] - 7s 19ms/step - loss: 0.0086 - accuracy: 0.9987
Epoch 8/10
350/350 [==============================] - 6s 19ms/step - loss: 0.0022 - accuracy: 0.9998
Epoch 9/10
350/350 [==============================] - 6s 18ms/step - loss: 4.5088e-04 - accuracy: 1.0000
Epoch 10/10
350/350 [==============================] - 6s 18ms/step - loss: 1.7073e-04 

In [56]:
y_pred=model.predict(X_test)
y_pred

array([[1.0000000e+00, 2.6092205e-18, 7.2434837e-23, 3.6349709e-20,
        1.2081877e-22],
       [4.1222569e-07, 9.9999964e-01, 2.9637119e-19, 5.9603578e-20,
        1.9745976e-18],
       [8.1285095e-10, 1.0000000e+00, 1.9011230e-11, 1.1112585e-12,
        3.8458719e-11],
       ...,
       [1.6522692e-06, 9.9999833e-01, 4.9274169e-18, 1.1843046e-18,
        2.5017763e-17],
       [1.0000000e+00, 1.1203753e-13, 4.6827845e-28, 9.7439606e-26,
        5.6280218e-27],
       [1.6581621e-09, 1.0000000e+00, 1.7502787e-25, 2.3695761e-26,
        5.0973971e-24]], dtype=float32)

In [57]:
y_pred=np.argmax(y_pred,axis=1)
y_pred

array([0, 1, 1, ..., 1, 0, 1], dtype=int64)

In [58]:
cm=confusion_matrix(y_test,y_pred)
print(cm)

[[6589 1011]
 [1009 6391]]


In [59]:
print(classification_report(y_test,y_pred))


              precision    recall  f1-score   support

           0       0.87      0.87      0.87      7600
           1       0.86      0.86      0.86      7400

    accuracy                           0.87     15000
   macro avg       0.87      0.87      0.87     15000
weighted avg       0.87      0.87      0.87     15000

